# Workflow for a multi-regional energy system

In this application of the FINE framework, a multi-regional energy system is modeled and optimized.

All classes which are available to the user are utilized and examples of the selection of different parameters within these classes are given.

The workflow is structures as follows:
1. Required packages are imported and the input data path is set
2. An energy system model instance is created
3. Commodity sources are added to the energy system model
4. Commodity conversion components are added to the energy system model
5. Commodity storages are added to the energy system model
6. Commodity transmission components are added to the energy system model
7. Commodity sinks are added to the energy system model
8. The energy system model is optimized
9. Selected optimization results are presented


# 1. Import required packages and set input data path

The FINE framework is imported which provides the required classes and functions for modeling the energy system.

In [1]:
import FINE as fn
from getData import getData
import pandas as pd
data = getData()

%load_ext autoreload
%autoreload 2

# 2. Create an energy system model instance 

The structure of the energy system model is given by the considered locations, commodities, the number of time steps as well as the hours per time step.

The commodities are specified by a unit (i.e. 'GW_electric', 'GW_H2lowerHeatingValue', 'Mio. t CO2/h') which can be given as an energy or mass unit per hour. Furthermore, the cost unit and length unit are specified.

In [2]:
locations = {'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'}
commodityUnitDict = {'electricity': 'GW_el', 'methane': 'GW_CH4_LHV', 'biogas': 'GW_biogas_LHV',
                     'CO2': 'Mio. t CO2/h', 'hydrogen': 'GW_H2_LHV'}
commodities = {'electricity', 'hydrogen', 'methane', 'biogas', 'CO2'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

In [3]:
esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                           commodityUnitsDict=commodityUnitDict,
                           hoursPerTimeStep=1, costUnit='1e9 Euro', lengthUnit='km')

In [4]:
CO2_reductionTarget = 0.85

# 3. Add commodity sources to the energy system model

## 3.1. Electricity sources

### Wind onshore

In [5]:
esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (onshore), operationRateMax'],
                  capacityMax=data['Wind (onshore), capacityMax'],
                  investPerCapacity=1.1, opexPerCapacity=1.1*0.02, interestRate=0.08,
                  economicLifetime=20))

Full load hours:

In [6]:
data['Wind (onshore), operationRateMax'].sum()

cluster_0    2300.406907
cluster_1    1082.032187
cluster_2    1447.812026
cluster_3    1448.827987
cluster_4    1935.618218
cluster_5    1757.696047
cluster_6    1286.944636
cluster_7    1199.380071
dtype: float64

### Wind offshore

In [7]:
esM.add(fn.Source(esM=esM, name='Wind (offshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (offshore), operationRateMax'],
                  capacityMax=data['Wind (offshore), capacityMax'],
                  investPerCapacity=2.3, opexPerCapacity=2.3*0.02, interestRate=0.08,
                  economicLifetime=20))

In [8]:
data['PV, capacityMax']

index
cluster_0    17.970141
cluster_1    25.807194
cluster_2    18.698614
cluster_3    32.994484
cluster_4    20.883876
cluster_5    39.127909
cluster_6    22.626522
cluster_7    21.891252
Name: 0, dtype: float64

In [9]:
for value in data['Existing run-of-river plants, capacityFix'].values:
    print(value)

0.0
0.544397347
0.455005165
0.037456259
0.0
0.043571567
2.075537801
0.655031861


Full load hours:

In [10]:
data['Wind (offshore), operationRateMax'].sum()

cluster_0    4332.977326
cluster_1       0.000000
cluster_2       0.000000
cluster_3       0.000000
cluster_4    3963.044612
cluster_5    3459.283279
cluster_6       0.000000
cluster_7       0.000000
dtype: float64

### PV

In [11]:
esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'], capacityMax=data['PV, capacityMax'],
                  investPerCapacity=0.65, opexPerCapacity=0.65*0.02, interestRate=0.08,
                  economicLifetime=25))

Full load hours:

In [12]:
data['PV, operationRateMax'].sum()

cluster_0    859.175855
cluster_1    970.535848
cluster_2    913.049274
cluster_3    917.880212
cluster_4    875.448945
cluster_5    904.590003
cluster_6    984.223517
cluster_7    992.311727
dtype: float64

### Exisisting run-of-river hydroelectricity plants

In [13]:
esM.add(fn.Source(esM=esM, name='Existing run-of-river plants', commodity='electricity',
                  hasCapacityVariable=True,
                  operationRateFix=data['Existing run-of-river plants, operationRateFix'],
                  capacityFix=data['Existing run-of-river plants, capacityFix'],
                  investPerCapacity=0, opexPerCapacity=0.208))

esM.add(fn.Source(esM=esM, name='Electricity purchase', commodity='electricity',
                  hasCapacityVariable=False, commodityCost=1))

## 3.2. Methane (natural gas and biogas)

### Natural gas

In [14]:
elig = pd.Series([1,0,0,0,0,0,0,0], index=locations)
esM.add(fn.Source(esM=esM, name='Natural gas purchase', commodity='methane',
                  hasCapacityVariable=False, commodityCost=0.0331*1e-3)) #locationalEligibility=elig,

### Biogas

In [15]:
esM.add(fn.Source(esM=esM, name='Biogas purchase', commodity='biogas',
                  operationRateMax=data['Biogas, operationRateMax'], hasCapacityVariable=False,
                  commodityCost=0.05409*1e-3))

## 3.3 CO2

### CO2

The CO2 source is required for the case in which it can be mixed with conventional natural gas.

In [16]:
esM.add(fn.Source(esM=esM, name='CO2 from enviroment', commodity='CO2',
                  hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 4. Add conversion components to the energy system model

### Biogas to methane

In [17]:
esM.add(fn.Conversion(esM=esM, name='Biogas to methane', physicalUnit='GW_biogas_LHV',
                      commodityConversionFactors={'biogas':-1, 'methane':1, 'CO2':-201*1e-6*0.625},
                      hasCapacityVariable=False))

### Existing combined cycle gas turbine plants

In [18]:
esM.add(fn.Conversion(esM=esM, name='Existing CCGT plants (methane)', physicalUnit='GW_el',
                      commodityConversionFactors={'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625},
                      hasCapacityVariable=True, capacityMax=data['Existing CCGT plants (methane), capacityMax'],
                      investPerCapacity=0, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### New combined cycle gas turbine plants for biogas

In [19]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (biogas)', physicalUnit='GW_el',
                      commodityConversionFactors={'electricity':1, 'biogas':-1/0.635},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### New combined cycly gas turbines for hydrogen

In [20]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)', physicalUnit='GW_el',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.927, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### Electrolyzers

In [21]:
esM.add(fn.Conversion(esM=esM, name='Electroylzers', physicalUnit='GW_el',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))

# 5. Add commodity storages to the energy system model

## 5.1. Electricity storage

### Lithium ion batteries

The self discharge of a lithium ion battery is here described as 3% per month. The self discharge per hours is obtained using the equation (1-$\text{selfDischarge}_\text{hour})^{30*24\text{h}} = 1-\text{selfDischarge}_\text{month}$.

In [22]:
esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.95,
                   cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                   chargeRate=1, dischargeRate=1, doPreciseTsaModeling=False,
                   investPerCapacity=0.151, opexPerCapacity=0.002, interestRate=0.08,
                   economicLifetime=22))

## 5.2. Hydrogen storage

### Hydrogen filled salt caverns
The maximum capacity is here obtained by: dividing the given capacity (which is given for methane) by the lower heating value of methane and then multiplying it with the lower heating value of hydrogen.

In [23]:
esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=133,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (hydrogen), capacityMax'],
                   investPerCapacity=0.00011, opexPerCapacity=0.00057, interestRate=0.08,
                   economicLifetime=30))

In [24]:
data['Salt caverns (hydrogen), capacityMax']

cluster_0     1235.216115
cluster_1        0.000000
cluster_2    11665.929973
cluster_3    11254.191268
cluster_4    19214.472897
cluster_5      274.492470
cluster_6        0.000000
cluster_7        0.000000
Name: MaxPotential_GWh, dtype: float64

## 5.3. Methane storage

### Methane filled salt caverns

In [25]:
esM.add(fn.Storage(esM=esM, name='Salt caverns (methane)', commodity='methane',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=443,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (methane), capacityMax'],
                   investPerCapacity=0.00004, opexPerCapacity=0.00001, interestRate=0.08,
                   economicLifetime=30))

## 5.4 Pumped hydro storage

### Pumped hydro storage

In [26]:
esM.add(fn.Storage(esM=esM, name='Pumped hydro storage', commodity='electricity',
                   chargeEfficiency=0.88, dischargeEfficiency=0.88,
                   hasCapacityVariable=True, selfDischarge=1-(1-0.00375)**(1/(30*24)),
                   chargeRate=0.16, dischargeRate=0.12, capacityFix=data['Pumped hydro storage, capacityFix'],
                   investPerCapacity=0, opexPerCapacity=0.000153))

In [27]:
1-(1-0.00375)**(1/(30*24))

5.2181098268988535e-06

# 6. Add commodity transmission components to the energy system model

## 6.1. Electricity transmission

### AC cables

esM.add(fn.LinearOptimalPowerFlow(esM=esM, name='AC cables', commodity='electricity',
                                  hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix'],
                                  reactances=data['AC cables, reactances']))

In [28]:
esM.add(fn.Transmission(esM=esM, name='AC cables', commodity='electricity',
                                  hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix']))

The distances of a component are set to a normalized values of 1.


### DC cables

In [29]:
esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', losses=data['DC cables, losses'],
                        distances=data['DC cables, distances'],
                        hasCapacityVariable=True, capacityFix=data['DC cables, capacityFix']))

In [30]:
esM.getComponentAttribute('DC cables','distances')

cluster_0_cluster_1    712.205608
cluster_0_cluster_7    776.355668
cluster_1_cluster_0    712.205608
cluster_1_cluster_2    285.575229
cluster_2_cluster_1    285.575229
cluster_2_cluster_4    272.406543
cluster_3_cluster_7    416.630462
cluster_4_cluster_2    272.406543
cluster_7_cluster_0    776.355668
cluster_7_cluster_3    416.630462
dtype: float64

## 6.2 Methane transmission

### Methane pipeline

In [31]:
esM.add(fn.Transmission(esM=esM, name='Pipelines (methane)', commodity='methane', 
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                        investPerCapacity=0.000037, investIfBuilt=0.000314,
                        interestRate=0.08, economicLifetime=40))

esM.getComponent('Pipelines (methane)')._mapC['cluster_0_cluster_4'][1]

## 6.3 Hydrogen transmission

### Hydrogen pipelines

In [32]:
esM.add(fn.Transmission(esM=esM, name='Pipelines (hydrogen)', commodity='hydrogen',
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                        investPerCapacity=0.000177, investIfBuilt=0.00033,
                        interestRate=0.08, economicLifetime=40))

esM.getComponent('Pipeline (hydrogen)')._mapC

# 7. Add commodity sinks to the energy system model

## 7.1. Electricity sinks

### Electricity demand

In [33]:
esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

## 7.2. Hydrogen sinks

### Fuel cell electric vehicle (FCEV) demand

In [34]:
FCEV_penetration=0.5
esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen', hasCapacityVariable=False,
                operationRateFix=data['Hydrogen demand, operationRateFix']*FCEV_penetration))

## 7.3. CO2 sinks

### CO2 exiting the system's boundary

In [35]:
esM.add(fn.Sink(esM=esM, name='CO2 to enviroment', commodity='CO2',
                hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 8. Optimize energy system model

All components are now added to the model and the model can be optimized. If the computational complexity of the optimization should be reduced, the time series data of the specified components can be clustered before the optimization and the parameter timeSeriesAggregation is set to True in the optimize call.

In [36]:
esM.cluster(numberOfTypicalPeriods=3)


Clustering time series data with 3 typical periods and 24 time steps per period...
		(3.8830 sec)



In [37]:
esM.optimize(timeSeriesAggregation=True, optimizationSpecs='LogToConsole=1 OptimalityTol=1e-3 cuts=0 method=2')

Time series aggregation specifications:
Number of typical periods: 3 , number of time steps per periods: 24
Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(1.6350 sec)
Declaring sets, variables and constraints for ConversionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0500 sec)
Declaring sets, variables and constraints for TransmissionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.1250 sec)
Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.1910 sec)
Declaring shared potential constraint...
		(0.0020 sec)
Declaring commodity balances...
		(0.3320 sec)
Declaring objective function...
		(0.2620 sec)
Academic license - for non-commercial use only
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Parameter LogToConsol

# 9. Selected results output

### Sources and Sink

In [38]:
esM.getOptimizationSummary("SourceSinkModel", outputLevel=2)

cluster_0  \
Component                    Property    Unit                            
Biogas purchase              TAC         [1e9 Euro/a]         0.268478   
                             commodCosts [1e9 Euro/a]         0.268478   
                             operation   [GW_biogas_LHV*h/a]   4963.54   
CO2 to enviroment            operation   [Mio. t CO2/h*h/a]          0   
Electricity demand           operation   [GW_el*h/a]           30957.9   
Existing run-of-river plants TAC         [1e9 Euro/a]              NaN   
                             capacity    [GW_el]                   NaN   
                             operation   [GW_el*h/a]               NaN   
                             opexCap     [1e9 Euro/a]              NaN   
Hydrogen demand              operation   [GW_H2_LHV*h/a]       4765.07   
Natural gas purchase         TAC         [1e9 Euro/a]                0   
                             commodCosts [1e9 Euro/a]                0   
                             operation   [GW_CH4_LHV*h/a]            0   
PV                           TAC         [1e9 Euro/a]          1.32784   
                             capacity    [GW_el]               17.9701   
                             capexCap    [1e9 Euro/a]          1.09422   
                             invest      [1e9 Euro]            11.6806   
                             operation   [GW_el*h/a]           15052.3   
                             opexCap     [1e9 Euro/a]         0.233612   
Wind (offshore)              TAC         [1e9 Euro/a]          4.51415   
                             capacity    [GW_el]                16.107   
                             capexCap    [1e9 Euro/a]          3.77323   
                             invest      [1e9 Euro]            37.0461   
                             operation   [GW_el*h/a]           53608.9   
                             opexCap     [1e9 Euro/a]         0.740923   
Wind (onshore)               TAC         [1e9 Euro/a]          7.79686   
                             capacity    [GW_el]               58.1693   
                             capexCap    [1e9 Euro/a]          6.51714   
                             invest      [1e9 Euro]            63.9862   
                             operation   [GW_el*h/a]            112412   
                             opexCap     [1e9 Euro/a]          1.27972   

                                                             cluster_1  \
Component                    Property    Unit                            
Biogas purchase              TAC         [1e9 Euro/a]         0.292266   
                             commodCosts [1e9 Euro/a]         0.292266   
                             operation   [GW_biogas_LHV*h/a]   5403.33   
CO2 to enviroment            operation   [Mio. t CO2/h*h/a]    4.22209   
Electricity demand           operation   [GW_el*h/a]            111621   
Existing run-of-river plants TAC         [1e9 Euro/a]         0.113235   
                             capacity    [GW_el]              0.544397   
                             operation   [GW_el*h/a]           2668.85   
                             opexCap     [1e9 Euro/a]         0.113235   
Hydrogen demand              operation   [GW_H2_LHV*h/a]       13481.3   
Natural gas purchase         TAC         [1e9 Euro/a]         0.695279   
                             commodCosts [1e9 Euro/a]         0.695279   
                             operation   [GW_CH4_LHV*h/a]      21005.4   
PV                           TAC         [1e9 Euro/a]          1.90692   
                             capacity    [GW_el]               25.8072   
                             capexCap    [1e9 Euro/a]          1.57143   
                             invest      [1e9 Euro]            16.7747   
                             operation   [GW_el*h/a]           25046.8   
                             opexCap     [1e9 Euro/a]         0.335494   
Wind (offshore)              TAC         [1e9 Euro/a]           

### Conversion

In [39]:
esM.getOptimizationSummary("ConversionModel", outputLevel=2)

cluster_0  cluster_1  \
Component                      Property  Unit                                 
Electroylzers                  TAC       [1e9 Euro/a]    1.48376          0   
                               capacity  [GW_el]         17.0518          0   
                               capexCap  [1e9 Euro/a]    1.27061          0   
                               invest    [1e9 Euro]       8.5259          0   
                               operation [GW_el*h/a]     95323.8          0   
                               opexCap   [1e9 Euro/a]   0.213147          0   
Existing CCGT plants (methane) TAC       [1e9 Euro/a]        NaN  0.0357861   
                               capacity  [GW_el]             NaN     1.7041   
                               operation [GW_el*h/a]         NaN    13128.4   
                               opexCap   [1e9 Euro/a]        NaN  0.0357861   
New CCGT plants (biogas)       TAC       [1e9 Euro/a]   0.049394  0.0446187   
                               capacity  [GW_el]        0.603867   0.545487   
                               capexCap  [1e9 Euro/a]  0.0367128  0.0331635   
                               invest    [1e9 Euro]     0.422707   0.381841   
                               operation [GW_el*h/a]     3151.85    3431.11   
                               opexCap   [1e9 Euro/a]  0.0126812  0.0114552   

                                                       cluster_2  cluster_3  \
Component                      Property  Unit                                 
Electroylzers                  TAC       [1e9 Euro/a]          0          0   
                               capacity  [GW_el]               0          0   
                               capexCap  [1e9 Euro/a]          0          0   
                               invest    [1e9 Euro]            0          0   
                               operation [GW_el*h/a]           0          0   
                               opexCap   [1e9 Euro/a]          0          0   
Existing CCGT plants (methane) TAC       [1e9 Euro/a]    0.06447        NaN   
                               capacity  [GW_el]            3.07        NaN   
                               operation [GW_el*h/a]     23651.3        NaN   
                               opexCap   [1e9 Euro/a]    0.06447        NaN   
New CCGT plants (biogas)       TAC       [1e9 Euro/a]  0.0433331  0.0682326   
                               capacity  [GW_el]         0.52977   0.834179   
                               capexCap  [1e9 Euro/a]  0.0322079  0.0507149   
                               invest    [1e9 Euro]     0.370839   0.583925   
                               operation [GW_el*h/a]     3225.24    5246.99   
                               opexCap   [1e9 Euro/a]  0.0111252  0.0175178   

                                                       cluster_4  cluster_5  \
Component                      Property  Unit                                 
Electroylzers                  TAC       [1e9 Euro/a]          0          0   
                               capacity  [GW_el]               0          0   
                               capexCap  [1e9 Euro/a]          0          0   
                               invest    [1e9 Euro]            0          0   
                               operation [GW_el*h/a]           0          0   
                               opexCap   [1e9 Euro/a]          0          0   
Existing CCGT plants (methane) TAC       [1e9 Euro/a]        NaN    0.02415   
                               capacity  [GW_el]             NaN       1.15   
                               operation [GW_el*h/a]         NaN     8859.6   
                               opexCap   [1e9 Euro/a]        NaN    0.02415   
New CCGT plants (biogas)       TAC       [1e9 Euro/a]  0.0600788  0.0652833   
                               capacity  [GW_el]        0.734495   0.798122   
                               capexCap  [1e9 Euro/a]  0.0446544  0.0485227   
                        

### Storage

In [40]:
esM.getOptimizationSummary("StorageModel", outputLevel=2).sum(axis=1)

Component                Property            Unit           
Li-ion batteries         TAC                 [1e9 Euro/a]           2.815857
                         capacity            [GW_el*h]            167.582186
                         capexCap            [1e9 Euro/a]           2.480693
                         invest              [1e9 Euro]            25.304910
                         operationCharge     [GW_el*h/a]        58044.499955
                         operationDischarge  [GW_el*h/a]        52356.532699
                         opexCap             [1e9 Euro/a]           0.335164
Pumped hydro storage     TAC                 [1e9 Euro/a]           0.009971
                         capacity            [GW_el*h]             65.173000
                         operationCharge     [GW_el*h/a]        13539.239352
                         operationDischarge  [GW_el*h/a]        10484.290903
                         opexCap             [1e9 Euro/a]           0.009971
Salt caverns (h

## Transmission

In [41]:
esM.getOptimizationSummary("TransmissionModel", outputLevel=2)

cluster_0  \
AC cables            capacity     [GW_el]         cluster_0         None   
                                                  cluster_1         None   
                                                  cluster_2         None   
                                                  cluster_3      2.39577   
                                                  cluster_4      3.26266   
                                                  cluster_5      2.39577   
                                                  cluster_6         None   
                                                  cluster_7         None   
                     operation    [GW_el*h/a]     cluster_0         None   
                                                  cluster_1         None   
                                                  cluster_2         None   
                                                  cluster_3      4873.45   
                                                  cluster_4      7617.49   
                                                  cluster_5      3750.99   
                                                  cluster_6         None   
                                                  cluster_7         None   
DC cables            capacity     [GW_el]         cluster_0         None   
                                                  cluster_1            2   
                                                  cluster_2         None   
                                                  cluster_3         None   
                                                  cluster_4         None   
                                                  cluster_7            2   
                     operation    [GW_el*h/a]     cluster_0         None   
                                                  cluster_1      257.622   
                                                  cluster_3         None   
                                                  cluster_4         None   
Pipelines (hydrogen) TAC          [1e9 Euro/a]    cluster_0         None   
                                                  cluster_1         None   
                                                  cluster_2         None   
                                                  cluster_3         None   
...                                                                  ...   
                     capexIfBuilt [1e9 Euro/a]    cluster_1         None   
                                                  cluster_2         None   
                                                  cluster_3         None   
                                                  cluster_4  1.38369e-05   
                                                  cluster_5         None   
                                                  cluster_6         None   
                                                  cluster_7         None   
                     invest       [1e9 Euro]      cluster_0         None   
                                                  cluster_1         None   
                                                  cluster_2         None   
                                                  cluster_3         None   
                                                  cluster_4   0.00112819   
                                                  cluster_5         None   
                                                  cluster_6         None   
                                                  cluster_7         None   
                     isBuilt      [-]             cluster_0         None   
                                                  cluster_1         None   
                                                  cluster_2         None   
                                                  cluster_3         None   
                                                  cluster_4            1   
                                                  cluster_5         None   
                                   

In [42]:
esM.componentModelingDict["TransmissionModel"].operationVariablesOptimum.head()

0         1         2         3     \
AC cables cluster_0 cluster_3  2.395773  2.395773  2.395773  2.395773   
                    cluster_4  3.262664  3.262664  3.262664  3.262664   
                    cluster_5  2.395773  2.395773  2.395773  2.395773   
          cluster_1 cluster_2  3.887811  3.574308  3.954215  4.120013   
                    cluster_3  0.372380  0.492154  0.000000  0.000000   

                                   4         5         6         7     \
AC cables cluster_0 cluster_3  2.395773  2.395773  2.395773  2.395773   
                    cluster_4  3.262664  3.262664  3.262664  3.262664   
                    cluster_5  2.395773  2.395773  2.395773  2.395773   
          cluster_1 cluster_2  4.245083  4.511573  5.874089  5.284760   
                    cluster_3  0.000000  0.000000  0.000000  0.000000   

                                   8         9       ...         8750  \
AC cables cluster_0 cluster_3  2.110190  2.395773    ...     2.395773   
                    cluster_4  3.262664  1.414562    ...     3.262664   
                    cluster_5  0.000000  0.000000    ...     2.395773   
          cluster_1 cluster_2  4.657870  8.516619    ...     4.478711   
                    cluster_3  0.000000  0.000000    ...     0.000000   

                                   8751      8752      8753      8754  \
AC cables cluster_0 cluster_3  2.395773  2.395773  2.395773  2.395773   
                    cluster_4  3.262664  3.262664  3.262664  3.262664   
                    cluster_5  2.395773  2.395773  2.395773  2.395773   
          cluster_1 cluster_2  2.738821  5.168040  1.908886  5.392371   
                    cluster_3  0.000000  0.000000  0.000000  0.000000   

                                   8755      8756      8757      8758  \
AC cables cluster_0 cluster_3  2.395773  2.395773  2.395773  2.395773   
                    cluster_4  3.262664  3.262664  3.262664  3.262664   
                    cluster_5  2.395773  2.395773  2.395773  2.395773   
          cluster_1 cluster_2  5.518450  4.797461  4.179286  3.454540   
                    cluster_3  0.000000  0.000000  0.000000  0.000000   

                                   8759  
AC cables cluster_0 cluster_3  2.395773  
                    cluster_4  3.262664  
                    cluster_5  2.395773  
          cluster_1 cluster_2  2.983000  
                    cluster_3  0.000000  

[5 rows x 8760 columns]

In [43]:
df=esM.componentModelingDict["TransmissionModel"].capacityVariablesOptimum
df.loc['Pipelines (methane)']+df.loc['Pipelines (hydrogen)']

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
cluster_0,NaN,NaN,NaN,NaN,10.883543,NaN,NaN,NaN
cluster_1,NaN,NaN,6.502254,NaN,NaN,NaN,NaN,3.218364
cluster_2,NaN,6.502254,NaN,NaN,6.975630,NaN,NaN,NaN
cluster_3,NaN,NaN,NaN,NaN,2.652877,1.337876,NaN,NaN
cluster_4,10.883543,NaN,6.975630,2.652877,NaN,NaN,NaN,NaN
cluster_5,NaN,NaN,NaN,1.337876,NaN,NaN,NaN,NaN
cluster_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.545445
cluster_7,NaN,3.218364,NaN,NaN,NaN,NaN,1.545445,NaN
